In [1]:
from keras.models import Sequential
from keras.layers import Dense, Recurrent, BatchNormalization, LSTM
from keras.regularizers import l2
from keras.callbacks import Callback
from pandas import DataFrame
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time
import os
from sklearn.model_selection import train_test_split


MODEL_DIR = './project/models/'

Using TensorFlow backend.
/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.20) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def print_samples(x, y, n=10):
    higher = len(x)
    print(higher)
    for i in range(n):
        j = np.random.randint(0, higher)
        print(x[j], y[j])

In [3]:
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data[:2]
        self.net_params = test_data[2:]
        self.epochs = [16, 32, 64, 128]

    def on_epoch_end(self, epoch, logs={}):
        if epoch in self.epochs:
            x, y = self.test_data
            h_l, n, t = self.net_params
            loss, acc = self.model.evaluate(x.reshape(len(x), 1,10), y, verbose=0)
            with open('architecture_comparison.txt', 'a+') as f:
                f.write('layers: {:}, neurons: {:}, epoch:  {:}, loss: {}, acc: {}\n'.format(h_l, n, epoch, loss, acc))

In [4]:
def split_data(x, y, ratio=0.2):
    # converting to numpy array
    x = np.array(x)
    y = np.array(y)
    
    # splitting dataset in training and testing
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = ratio)
    return x_train, y_train, x_test, y_test

def build_model(x_train, y_train, x_test, y_test, target, hidden_layers=2, neurons=32, epochs=10, plot=True, model_path=None):

    model = Sequential()
    # model.add(Dense(64, input_shape = (1, 10)))
    # normalizing the input
    model.add(BatchNormalization(input_shape=(1,10)))
    
    for i in range(hidden_layers - 1):
        model.add(LSTM(neurons, dropout=0.3, recurrent_dropout=0.2, return_sequences = True))
    model.add(LSTM(neurons, dropout=0.3, recurrent_dropout=0.2))
    
    model.add(Dense(10, activation='sigmoid', kernel_regularizer=l2(0.01)))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
    metrics=['categorical_accuracy'])

    # default batch size = 32
    t0 = time.time()
    
    history = model.fit(x_train.reshape(len(x_train),1,10), y_train, validation_split=0.15, epochs=epochs, verbose=0) 
#     history = model.fit(x_train.reshape(len(x_train),1,10), y_train, validation_split=0.15, epochs=epochs, verbose=0, 
#                         callbacks=[TestCallback((x_test, y_test, hidden_layers, neurons, target))]) 
    t1 = time.time()
    
    if model_path:
        model.save(model_path)
        
    if plot:
        #print(history.history.keys())
        os.makedirs('plots/{:}'.format(target), exist_ok=True)
        
        plt.figure(figsize=(15, 5))
        #  "Accuracy"
        plt.subplot(121)
        plt.plot(history.history['categorical_accuracy'])
        plt.plot(history.history['val_categorical_accuracy'])
        plt.grid()
        plt.title('Model Accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['Training data', 'Validation data'], loc='lower right')
        # "Loss"
        plt.subplot(122)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.grid()
        plt.title('Model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['Training data', 'Validation data'], loc='upper right')
        plt.savefig('plots/{:}/plot_{:}_{:}_{:}.pdf'.format(target, hidden_layers, neurons, epochs))
        plt.close()
        
#     print('Evaluating model')
    loss, acc = model.evaluate(x_test.reshape(len(x_test), 1,10), y_test, verbose=0)
#     with open('architecture_comparison.txt', 'a+') as model_f:
#         model_f.write('layers: {:}, neurons: {:}, epoch:  {:}, loss: {}, acc: {}, time: {:}\n'.format(
#             hidden_layers, neurons, epochs, loss, acc, t1-t0))
#      print('\n')
#     with open('./model_cmp_time_norm.txt', 'a+') as model_f:
#         model_f.write('hidden layers: {:}, neurons: {:}, epochs: {:}, time: {:}'
#                 .format(hidden_layers, neurons, epochs, t1-t0))
#         for i in range(len(model.metrics_names)):
#             model_f.write(" " + str(model.metrics_names[i]) + ": " + str(metrics[i]))
#         model_f.write('\n')
    
    return history.history, [loss, acc]


In [5]:
layers = 4
neurons = 32
train_epochs = 150

for target in os.listdir(MODEL_DIR):
    path = os.path.join(MODEL_DIR, target)
    print(target)
    train_hist = []
    test_metrics = []
    for router in os.listdir(path):
        model = os.path.join(path, router)
        x = []
        y = []
        for dataset in os.listdir(model):
            file = os.path.join(model, dataset)
            with open(file, 'r') as f:
                lines = f.readlines()
            for line in lines:    
                cnt, label = line.split(',')
                cnt = np.array([int(c) for c in cnt.split()[1:]], dtype=np.int)
                label = np.array([int(l) for l in label.split()], dtype=np.int)
                x.append(cnt)
                y.append(label)
        
        x_t, y_t, x_ts, y_ts = split_data(x, y)
        h, m = build_model(x_t, y_t, x_ts, y_ts, target, neurons=neurons, hidden_layers=layers, epochs=train_epochs,
                           plot=False, model_path=os.path.join(model,'{:}_model.h5'.format(router)))
        print(model, m)
        train_hist.append(h)
        test_metrics.append(m)
     
        
    with open(os.path.join(path, 'train_hist.pkl'), 'wb') as th:
        pickle.dump(train_hist, th)
        
    with open(os.path.join(path, 'test_metrics.pkl'), 'wb') as tm:
        pickle.dump(test_metrics, tm)

r1_172_168_35_1
./project/models/r1_172_168_35_1/r7/r7_model.h5
./project/models/r1_172_168_35_1/r7 [0.0008521629507880086, 1.0]


KeyboardInterrupt: 

In [ ]:
# CELL FOR ARCHITECTURE TESTING

# layers = [2, 4, 6, 8]
# neurons = [4, 8, 16, 32, 64, 128]
# epochs = [16, 32, 64, 128]
# train_epochs = 150
# train_hist = {}
# test_metrics = {}

# architecture_results = {}
# architecture_history = {}

# for target in os.listdir(MODEL_DIR):
#     path = os.path.join(MODEL_DIR, target)
#     print(target)
    
#     for router in os.listdir(path):
#         model = os.path.join(path, router)
#         x = []
#         y = []
#         for dataset in os.listdir(model):
#             file = os.path.join(model, dataset)
#             with open(file, 'r') as f:
#                 lines = f.readlines()
#             for line in lines:    
#                 cnt, label = line.split(',')
#                 cnt = np.array([int(c) for c in cnt.split()[1:]], dtype=np.int)
#                 label = np.array([int(l) for l in label.split()], dtype=np.int)
#                 x.append(cnt)
#                 y.append(label)
        
#         # running 10 times to find the best architecture
#         for k in range(10):
#             x_t, y_t, x_ts, y_ts = split_data(x, y)

#             for layer in layers:
#                 for neuron in neurons:
#                     h, m = build_model(x_t, y_t, x_ts, y_ts, target, neurons=neuron, hidden_layers=layer, epochs=train_epochs, plot=False)
#                     #train_hist[target]=h
#                     #test_metrics[target]=m
#                     key = '{:}_{:}'.format(layer, neuron)
#                     if key not in architecture_results:
#                         architecture_results[key] = []
#                         architecture_history[key] = []
#                     architecture_results[key].append(m)
#                     architecture_history[key].append(h)
        
#         with open('arch_hist.pkl', 'wb') as ah:
#             pickle.dump(architecture_history, ah)
        
#         with open('arch_metrics.pkl', 'wb') as am:
#             pickle.dump(architecture_results, am)        
        
# #     with open('train_hist.pkl', 'wb') as th:
# #         pickle.dump(train_hist, th)
        
# #     with open('test_metrics.pkl', 'wb') as tm:
# #         pickle.dump(test_metrics, tm)